In [72]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

Download data from kaggle

In [73]:
#!mkdir ~/.kaggle
#!cp kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets download -d lakshmi25npathi/online-retail-dataset

Extract Data

In [80]:
#!unzip online-retail-dataset.zip
df=pd.read_excel("online_retail.xlsx")

In [87]:
df
# df.info()
# df.describe()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530,United Kingdom


Data type casting

In [82]:
def auto_data_type(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        min_value = df[col].min()
        max_value = df[col].max()

        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype(np.float32)
        elif min_value >= 0:
            if max_value <= 255:
                df[col] = df[col].astype(np.uint8)
            elif max_value <= 65535:
                df[col] = df[col].astype(np.uint16)
            elif max_value <= 4294967295:
                df[col] = df[col].astype(np.uint32)
            else:
                df[col] = df[col].astype(np.uint64)
        else:
            if -128 <= min_value and max_value <= 127:
                df[col] = df[col].astype(np.int8)
            elif -32768 <= min_value and max_value <= 32767:
                df[col] = df[col].astype(np.int16)
            elif -2147483648 <= min_value and max_value <= 2147483647:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
    return df


clean_df=auto_data_type(df)
#df.info()


drop rows with missing Customer ID

In [88]:
df = df.dropna(subset=["Customer ID"])
df["Customer ID"] = df["Customer ID"].astype(np.uint16)

Remove rows with duplicate entries, missing and negative Quantity

In [89]:
df_copy=df.copy()
df_copy = df_copy.drop_duplicates()
df_copy = df_copy[df_copy["Quantity"] >= 0]
df_copy.info()


<class 'pandas.core.frame.DataFrame'>
Index: 400947 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      400947 non-null  object        
 1   StockCode    400947 non-null  object        
 2   Description  400947 non-null  object        
 3   Quantity     400947 non-null  int16         
 4   InvoiceDate  400947 non-null  datetime64[ns]
 5   Price        400947 non-null  float32       
 6   Customer ID  400947 non-null  uint16        
 7   Country      400947 non-null  object        
dtypes: datetime64[ns](1), float32(1), int16(1), object(4), uint16(1)
memory usage: 21.4+ MB


Encode Country column to numeric labels

In [90]:
le = LabelEncoder()
df_copy["Country_Code"] = le.fit_transform(df_copy["Country"].astype(str)).astype(np.uint8)

In [91]:
df_copy = df_copy.drop(columns=["Country"])

In [92]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400947 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Invoice       400947 non-null  object        
 1   StockCode     400947 non-null  object        
 2   Description   400947 non-null  object        
 3   Quantity      400947 non-null  int16         
 4   InvoiceDate   400947 non-null  datetime64[ns]
 5   Price         400947 non-null  float32       
 6   Customer ID   400947 non-null  uint16        
 7   Country_Code  400947 non-null  uint8         
dtypes: datetime64[ns](1), float32(1), int16(1), object(3), uint16(1), uint8(1)
memory usage: 18.7+ MB
